#                                     Text classification

In [1]:
#This project has two part in one first part we convert the data in required format i.e in form of 2D numpy array
#And then used the sklearn Navie Bayes.
#In second part we have implemented our own Navie Bayes
#It might take some Time to run the project because we are doing lot of work here.

In [2]:
#We have imported different libraries
#for importing the data we have used sklearn datasets
#Numpy array is imported so that we can create the Numpy 2D array in later half of this project
#Instead of using split() function we have word_tokenize from nltk because it is much easier to seprate each word easily
#Stop words are imported from  nltk.corpus and from string.puctuations we have imported the punctuation
#combining these two list of stop words we have made a final list of stop words

In [3]:
from sklearn import datasets
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords

In [4]:
#fetching the data from sklearn 
news = datasets.fetch_20newsgroups()

In [5]:
#seprating the data value and target 
X=news.data
Y=news.target

In [6]:
#import the stop word from nltk.corpus from those stop words we have only choose the word that are present in englist
stop=stopwords.words('english')
#since the above list of stop words does not contain he punctuations therfore importing punctuations from sting.puctuation
#And coverting it in form of list() so that we can prepare final list of stop words
pun=list(string.punctuation)
#creating the final list of Stop words
stop=stop+pun

In [7]:
#Splitting the Data in X_train,Y_train,X_test,Y_test
#if you want you can choose the test_size and random_state i have not chossen it here
from sklearn import model_selection
X_train,X_test,Y_train,Y_test=model_selection.train_test_split(X,Y)

In [8]:
#This function is used to create the frequency dictionary or you can say that Vocabulary that serve as your Feature after some modification
#One thing that is to remember is never serve your testing data to it because testing data has no connection with making of Vocabulary
#this function takes the X i.e documents in newsgroups and stop word list which we have made earlier
def frequency_dict(X,stop):
#here we are creating the list of words that are present in the documents for that we have taken a empty list.
    words=[]
#we are going to each document one by one.    
    for i in X:
#we are using word tokenize to split the words and appending them in a list
        val=word_tokenize(i)
        words.append(val)
#Now we are creating the fresh list of word which are not present in stop as well as there are serval other criteria listed below
#for this we have created a Empty list input_words
    input_words=[]
#so from word list we are going to each doc and in each doc we are going to each word
    for doc in words:
        for word in doc:
#here we are checking if that particular word is present in stop list or not.
#word.lower() is used because stop word list only contain word in lower chacter so either this For or FOR both of it are of no use because they are stop words
#Moreover words with length greater that 2 are of no use because they generally contain some special character(find by observation)
#some other stop criteria is also included which are not present in stop list
            if (word.lower() not in stop) and len(word)!=1 and len(word)!=2 and word[0]!="'" and word!="n't" and word[0]!=".":
                input_words.append(word)
#Now once the list we have we are making a dictionary of Vocabulary i.e. which word comes how many times for this we have created a dictionary
    vocab={}
#we go through each word of list if that word is present in dictionary increase its frequency by one 
#else if word is not present in dictionary then add the word and intialise its frequency by 1
    for i in input_words:
        if i in vocab:
            vocab[i]+=1
        else:
            vocab[i]=1
#we are returning this vocabulary
    return vocab

In [9]:
#simply calling the above function on training data
#we get the vocabulary required
Vocab=frequency_dict(X_train,stop)

In [10]:
#here we are creating the list of feature we have choose the words which have frequency greater thatn 20 but less that 10500
#we are doing this to avoid unneccesary words you can change it also but choosing this range will give you decent accuracy
#I have find out this using trail and error and it is so because word with lower frequency than 20 are occuring very less time
#There value are mostly 0 in numpy 2D array that we will create later so they does not affect much in classsification
#More over the words which are coming more than 10500 times are generally stop words which are not present in stop list so we ignore them also
#here we have created the feature list
features=[]
#we are going through each keys of Vocab and checking if that key frequency is in our range we add that word in our feature list
#Else ignore that word
for word in Vocab:
    if Vocab[word]>=20 and Vocab[word]<=10500:
        features.append(word)

In [11]:
#Here we are checking the length of feature list
len(features)

9227

In [12]:
#This function serve a very important purpose or you can say a most important function of the project
#This function modifies the data in required format i.e a Numpy 2D array format
#This function takes the Input data and the feature list and return a Numpy 2D array 
def modifier(X_data,features):
#Firstly we create a Numpy 2D array of required format and initially all entries are 0
#For Creating a Numpy array of required Format we are using np.zeros function and giving the shape len(X_data) is Number of rows and len(features) is Number of columns 
    modified_data=np.zeros(shape=(len(X_data),len(features)))
#Since this fuction takes some time to create a Numpy 2D array and sometimes it becomes quiet annoying to wait for the result.
#so we have created two variables which keep track of how much or what percentage of Numpy 2D array is created
#It is done simply done by increasing the count by 1 after each document and dividing it by total number of docs and multiplying by 100
#So we Intialises the variable accordingly
    max_count=len(X_data)
    count=0
#We are itterating for each document
    for i in range(len(X_data)):
#current_doc contains the current document on which we are iterating
        current_doc=X_data[i]
#current_doc_modified containg the list of words in that document so that in later part of the function we check that wether the word present in feature list or not
        current_doc_modified=word_tokenize(current_doc)
#we are creating dictionary to keep the track record of words and their frequencies in current doc
        d=dict()
#this dictionary contains the frequency of all the elements in our current_doc
#We are going to each word and checking if the word is present in dictionary then increase its frequency by 1
#else if the word is not part of dictionary make it the part of dictionary and intialise its frequency by one
        for word in current_doc_modified:
            if word in d.keys():
                d[word]+=1
            else:
                d[word]=1
        #dictionary created
#Now we itterating over the feture list which is passed at the starting of the function
        for j in range(len(features)):
#Checking if the feature is present in dictionary we are created
#if the Feature is present then replace 0 with that feature frequency.
#if the feature is not present then simply continue i.e 0 is present for that particular feature or word
            if features[j] in d.keys():
                modified_data[i][j]=d[features[j]]
            else:
                continue
#here we are just checking the progress noting else if it is not a part of function then also the work works perfectly fine 
        count+=1
        print((count/max_count)*100)
#here we are returning the numpy 2D array
    return modified_data
                

In [13]:
#Calling the function for training data So that we get the Training data in Numpy 2D array to train our model
X_train_modified=modifier(X_train,features)

0.011785503830288745
0.02357100766057749
0.03535651149086624
0.04714201532115498
0.05892751915144372
0.07071302298173247
0.08249852681202122
0.09428403064230996
0.1060695344725987
0.11785503830288745
0.1296405421331762
0.14142604596346495
0.15321154979375368
0.16499705362404243
0.1767825574543312
0.18856806128461992
0.20035356511490868
0.2121390689451974
0.22392457277548616
0.2357100766057749
0.24749558043606362
0.2592810842663524
0.27106658809664114
0.2828520919269299
0.29463759575721865
0.30642309958750735
0.3182086034177961
0.32999410724808487
0.3417796110783736
0.3535651149086624
0.3653506187389511
0.37713612256923984
0.3889216263995286
0.40070713022981735
0.41249263406010606
0.4242781378903948
0.4360636417206835
0.4478491455509723
0.4596346493812611
0.4714201532115498
0.48320565704183854
0.49499116087212724
0.506776664702416
0.5185621685327048
0.5303476723629935
0.5421331761932823
0.553918680023571
0.5657041838538598
0.5774896876841484
0.5892751915144373
0.601060695344726
0.612846

5.385975250441956
5.397760754272245
5.409546258102534
5.421331761932822
5.433117265763111
5.4449027695934005
5.456688273423689
5.468473777253978
5.480259281084266
5.492044784914555
5.503830288744844
5.515615792575133
5.527401296405421
5.53918680023571
5.550972304065999
5.562757807896287
5.574543311726576
5.586328815556866
5.598114319387154
5.609899823217442
5.621685327047731
5.63347083087802
5.645256334708309
5.657041838538598
5.668827342368886
5.680612846199175
5.692398350029464
5.704183853859752
5.715969357690041
5.727754861520331
5.739540365350618
5.751325869180907
5.7631113730111965
5.774896876841485
5.786682380671774
5.798467884502063
5.810253388332351
5.82203889216264
5.833824395992929
5.845609899823217
5.8573954036535065
5.869180907483795
5.880966411314083
5.892751915144372
5.9045374189746616
5.91632292280495
5.928108426635239
5.939893930465527
5.951679434295816
5.963464938126105
5.975250441956394
5.987035945786682
5.998821449616972
6.01060695344726
6.022392457277548
6.034177961

10.79552150854449
10.80730701237478
10.819092516205068
10.830878020035357
10.842663523865644
10.854449027695933
10.866234531526223
10.878020035356512
10.889805539186801
10.901591043017088
10.913376546847378
10.925162050677667
10.936947554507956
10.948733058338243
10.960518562168533
10.972304065998822
10.98408956982911
10.995875073659398
11.007660577489688
11.019446081319977
11.031231585150266
11.043017088980553
11.054802592810843
11.066588096641132
11.07837360047142
11.090159104301708
11.101944608131998
11.113730111962287
11.125515615792574
11.137301119622864
11.149086623453153
11.160872127283442
11.172657631113731
11.184443134944019
11.196228638774308
11.208014142604595
11.219799646434884
11.231585150265174
11.243370654095463
11.255156157925752
11.26694166175604
11.278727165586329
11.290512669416618
11.302298173246907
11.314083677077196
11.325869180907484
11.337654684737773
11.34944018856806
11.36122569239835
11.373011196228639
11.384796700058928
11.396582203889217
11.408367707719504


16.075427224513845
16.087212728344134
16.098998232174424
16.110783736004713
16.122569239835002
16.13435474366529
16.14614024749558
16.15792575132587
16.16971125515616
16.181496758986448
16.193282262816737
16.205067766647023
16.216853270477312
16.2286387743076
16.24042427813789
16.25220978196818
16.26399528579847
16.275780789628758
16.287566293459047
16.299351797289336
16.311137301119626
16.32292280494991
16.3347083087802
16.34649381261049
16.358279316440775
16.370064820271065
16.381850324101354
16.393635827931643
16.405421331761932
16.41720683559222
16.42899233942251
16.4407778432528
16.45256334708309
16.464348850913378
16.476134354743664
16.487919858573953
16.499705362404242
16.51149086623453
16.52327637006482
16.53506187389511
16.5468473777254
16.55863288155569
16.570418385385977
16.582203889216263
16.593989393046552
16.60577489687684
16.61756040070713
16.629345904537416
16.641131408367706
16.652916912197995
16.664702416028284
16.676487919858573
16.688273423688862
16.70005892751915
1

21.30819092516205
21.31997642899234
21.33176193282263
21.34354743665292
21.355332940483205
21.367118444313494
21.378903948143783
21.390689451974072
21.402474955804358
21.414260459634647
21.426045963464937
21.437831467295226
21.449616971125515
21.461402474955804
21.473187978786093
21.484973482616383
21.49675898644667
21.50854449027696
21.52032999410725
21.532115497937536
21.543901001767825
21.555686505598114
21.567472009428403
21.579257513258693
21.59104301708898
21.60282852091927
21.61461402474956
21.626399528579846
21.638185032410135
21.649970536240424
21.661756040070713
21.673541543901003
21.68532704773129
21.697112551561577
21.708898055391867
21.720683559222156
21.732469063052445
21.744254566882734
21.756040070713023
21.767825574543313
21.779611078373602
21.79139658220389
21.803182086034177
21.814967589864466
21.826753093694755
21.838538597525044
21.850324101355334
21.862109605185623
21.873895109015912
21.885680612846198
21.897466116676487
21.909251620506776
21.921037124337065
21.93

...

75.95757218621097
75.96935769004125
75.98114319387153
75.99292869770183
76.00471420153212
76.0164997053624
76.02828520919269
76.04007071302298
76.05185621685328
76.06364172068356
76.07542722451385
76.08721272834413
76.09899823217444
76.11078373600472
76.122569239835
76.13435474366528
76.14614024749558
76.15792575132588
76.16971125515616
76.18149675898644
76.19328226281674
76.20506776664703
76.21685327047732
76.2286387743076
76.24042427813788
76.25220978196819
76.26399528579847
76.27578078962875
76.28756629345904
76.29935179728933
76.31113730111963
76.32292280494991
76.3347083087802
76.34649381261049
76.35827931644079
76.37006482027107
76.38185032410135
76.39363582793163
76.40542133176194
76.41720683559222
76.4289923394225
76.44077784325279
76.45256334708309
76.46434885091338
76.47613435474366
76.48791985857395
76.49970536240424
76.51149086623454
76.52327637006482
76.5350618738951
76.5468473777254
76.5586328815557
76.57041838538598
76.58220388921626
76.59398939304656
76.60577489687684
7

81.80318208603417
81.81496758986447
81.82675309369476
81.83853859752504
81.85032410135533
81.86210960518562
81.87389510901592
81.8856806128462
81.89746611667648
81.90925162050678
81.92103712433708
81.93282262816736
81.94460813199764
81.95639363582792
81.96817913965822
81.97996464348851
81.9917501473188
82.00353565114908
82.01532115497938
82.02710665880967
82.03889216263995
82.05067766647024
82.06246317030053
82.07424867413083
82.08603417796111
82.09781968179139
82.10960518562167
82.12139068945199
82.13317619328227
82.14496169711255
82.15674720094283
82.16853270477313
82.18031820860342
82.1921037124337
82.20388921626399
82.21567472009428
82.22746022392458
82.23924572775486
82.25103123158515
82.26281673541543
82.27460223924574
82.28638774307602
82.2981732469063
82.30995875073658
82.32174425456688
82.33352975839718
82.34531526222746
82.35710076605774
82.36888626988804
82.38067177371833
82.39245727754862
82.4042427813789
82.4160282852092
82.42781378903949
82.43959929286977
82.4513847967000

87.25987035945786
87.27165586328816
87.28344136711844
87.29522687094872
87.30701237477902
87.31879787860932
87.3305833824396
87.34236888626988
87.35415439010018
87.36593989393046
87.37772539776076
87.38951090159104
87.40129640542133
87.41308190925162
87.42486741308191
87.4366529169122
87.44843842074249
87.46022392457277
87.47200942840307
87.48379493223335
87.49558043606365
87.50736593989393
87.51915144372421
87.53093694755451
87.54272245138479
87.55450795521509
87.56629345904537
87.57807896287567
87.58986446670595
87.60164997053624
87.61343547436653
87.62522097819682
87.6370064820271
87.6487919858574
87.66057748968768
87.67236299351796
87.68414849734826
87.69593400117856
87.70771950500884
87.71950500883912
87.73129051266942
87.74307601649971
87.75486152033
87.76664702416028
87.77843252799057
87.79021803182087
87.80200353565115
87.81378903948143
87.82557454331172
87.83736004714201
87.84914555097231
87.86093105480259
87.87271655863287
87.88450206246317
87.89628756629347
87.90807307012375

92.72834413671185
92.74012964054214
92.75191514437242
92.7637006482027
92.77548615203301
92.7872716558633
92.79905715969358
92.81084266352386
92.82262816735415
92.83441367118445
92.84619917501473
92.85798467884501
92.86977018267531
92.8815556865056
92.89334119033589
92.90512669416617
92.91691219799645
92.92869770182676
92.94048320565705
92.95226870948733
92.96405421331761
92.9758397171479
92.9876252209782
92.99941072480848
93.01119622863877
93.02298173246906
93.03476723629936
93.04655274012964
93.05833824395992
93.07012374779022
93.08190925162052
93.0936947554508
93.10548025928108
93.11726576311136
93.12905126694166
93.14083677077195
93.15262227460224
93.16440777843252
93.17619328226282
93.18797878609311
93.1997642899234
93.21154979375368
93.22333529758397
93.23512080141427
93.24690630524455
93.25869180907483
93.27047731290513
93.28226281673541
93.29404832056571
93.30583382439599
93.31761932822629
93.32940483205657
93.34119033588686
93.35297583971715
93.36476134354744
93.37654684737772

98.14967589864466
98.16146140247496
98.17324690630525
98.18503241013553
98.19681791396582
98.20860341779611
98.2203889216264
98.23217442545669
98.24395992928697
98.25574543311727
98.26753093694755
98.27931644077785
98.29110194460813
98.30288744843843
98.31467295226871
98.326458456099
98.33824395992929
98.35002946375958
98.36181496758986
98.37360047142015
98.38538597525044
98.39717147908074
98.40895698291102
98.4207424867413
98.4325279905716
98.44431349440188
98.45609899823218
98.46788450206246
98.47967000589276
98.49145550972304
98.50324101355334
98.51502651738362
98.5268120212139
98.5385975250442
98.55038302887449
98.56216853270477
98.57395403653506
98.58573954036535
98.59752504419565
98.60931054802593
98.62109605185621
98.63288155568651
98.6446670595168
98.65645256334709
98.66823806717737
98.68002357100765
98.69180907483795
98.70359457866824
98.71538008249853
98.72716558632881
98.7389510901591
98.7507365939894
98.76252209781968
98.77430760164997
98.78609310548026
98.79787860931056
98

In [14]:
#we are calling the modifier function to get our test Data in specified format
X_test_modified=modifier(X_test,features)

0.03534817956875221
0.07069635913750442
0.10604453870625664
0.14139271827500885
0.17674089784376104
0.2120890774125133
0.24743725698126548
0.2827854365500177
0.3181336161187699
0.3534817956875221
0.38882997525627433
0.4241781548250266
0.45952633439377877
0.49487451396253096
0.5302226935312832
0.5655708731000354
0.6009190526687875
0.6362672322375398
0.671615411806292
0.7069635913750442
0.7423117709437964
0.7776599505125487
0.8130081300813009
0.8483563096500532
0.8837044892188053
0.9190526687875575
0.9544008483563097
0.9897490279250619
1.025097207493814
1.0604453870625663
1.0957935666313185
1.1311417462000708
1.166489925768823
1.201838105337575
1.2371862849063273
1.2725344644750796
1.3078826440438318
1.343230823612584
1.378579003181336
1.4139271827500883
1.4492753623188406
1.4846235418875928
1.519971721456345
1.5553199010250973
1.5906680805938493
1.6260162601626018
1.661364439731354
1.6967126193001063
1.7320607988688583
1.7674089784376106
1.8027571580063628
1.838105337575115
1.8734535171

16.542948038176032
16.578296217744786
16.613644397313536
16.64899257688229
16.684340756451043
16.719688936019793
16.755037115588546
16.7903852951573
16.82573347472605
16.861081654294804
16.896429833863554
16.931778013432307
16.96712619300106
17.00247437256981
17.037822552138564
17.073170731707318
17.108518911276068
17.14386709084482
17.17921527041357
17.214563449982325
17.24991162955108
17.28525980911983
17.320607988688582
17.355956168257336
17.391304347826086
17.42665252739484
17.46200070696359
17.497348886532343
17.532697066101097
17.568045245669847
17.6033934252386
17.638741604807354
17.674089784376104
17.709437963944858
17.744786143513608
17.78013432308236
17.815482502651115
17.850830682219865
17.88617886178862
17.921527041357372
17.956875220926122
17.992223400494876
18.027571580063626
18.06291975963238
18.098267939201133
18.133616118769883
18.168964298338636
18.20431247790739
18.23966065747614
18.275008837044894
18.310357016613644
18.345705196182397
18.38105337575115
18.4164015553

32.7677624602333
32.80311063980205
32.8384588193708
32.87380699893955
32.90915517850831
32.94450335807706
32.97985153764581
33.015199717214564
33.050547896783314
33.085896076352064
33.121244255920814
33.15659243548957
33.19194061505832
33.22728879462707
33.26263697419583
33.29798515376458
33.33333333333333
33.368681512902086
33.404029692470836
33.439377872039586
33.47472605160834
33.51007423117709
33.54542241074584
33.5807705903146
33.61611876988335
33.6514669494521
33.68681512902085
33.72216330858961
33.75751148815836
33.79285966772711
33.828207847295864
33.863556026864615
33.898904206433365
33.93425238600212
33.96960056557087
34.00494874513962
34.04029692470838
34.07564510427713
34.11099328384588
34.146341463414636
34.181689642983386
34.217037822552136
34.252386002120886
34.28773418168964
34.32308236125839
34.35843054082714
34.3937787203959
34.42912689996465
34.4644750795334
34.49982325910216
34.53517143867091
34.57051961823966
34.605867797808415
34.641215977377165
34.676564156945915

49.275362318840585
49.310710498409335
49.346058677978085
49.381406857546835
49.41675503711559
49.45210321668434
49.48745139625309
49.52279957582185
49.5581477553906
49.59349593495935
49.62884411452811
49.66419229409686
49.69954047366561
49.73488865323436
49.770236832803114
49.805585012371864
49.840933191940614
49.876281371509364
49.91162955107812
49.94697773064687
49.98232591021562
50.01767408978438
50.05302226935313
50.08837044892188
50.123718628490636
50.159066808059386
50.194414987628136
50.22976316719689
50.26511134676564
50.30045952633439
50.33580770590314
50.37115588547189
50.40650406504065
50.4418522446094
50.47720042417816
50.51254860374691
50.54789678331566
50.58324496288441
50.618593142453165
50.653941322021915
50.68928950159067
50.72463768115942
50.75998586072817
50.79533404029692
50.83068221986567
50.86603039943442
50.90137857900318
50.936726758571936
50.972074938140686
51.007423117709436
51.042771297278186
51.078119476846936
51.11346765641569
51.14881583598445
51.184164015

65.7476139978791
65.78296217744786
65.81831035701661
65.85365853658537
65.88900671615411
65.92435489572287
65.95970307529161
65.99505125486037
66.03039943442913
66.06574761399789
66.10109579356663
66.13644397313539
66.17179215270413
66.20714033227289
66.24248851184163
66.2778366914104
66.31318487097914
66.3485330505479
66.38388123011664
66.4192294096854
66.45457758925414
66.48992576882291
66.52527394839166
66.56062212796041
66.59597030752916
66.63131848709791
66.66666666666666
66.70201484623541
66.73736302580417
66.77271120537293
66.80805938494167
66.84340756451043
66.87875574407917
66.91410392364793
66.94945210321669
66.98480028278544
67.02014846235419
67.05549664192294
67.09084482149169
67.12619300106044
67.1615411806292
67.19688936019796
67.2322375397667
67.26758571933546
67.3029338989042
67.33828207847296
67.3736302580417
67.40897843761047
67.44432661717921
67.47967479674797
67.51502297631671
67.55037115588547
67.58571933545421
67.62106751502299
67.65641569459173
67.69176387416049


82.07847295864264
82.11382113821138
82.14916931778014
82.18451749734889
82.21986567691764
82.25521385648639
82.29056203605515
82.32591021562389
82.36125839519265
82.39660657476139
82.43195475433015
82.4673029338989
82.50265111346765
82.5379992930364
82.57334747260516
82.6086956521739
82.64404383174266
82.67939201131142
82.71474019088016
82.75008837044892
82.78543655001768
82.82078472958642
82.85613290915518
82.89148108872394
82.92682926829268
82.96217744786144
82.99752562743019
83.03287380699894
83.06822198656769
83.10357016613644
83.13891834570519
83.17426652527395
83.2096147048427
83.24496288441145
83.28031106398021
83.31565924354895
83.35100742311771
83.38635560268645
83.42170378225522
83.45705196182396
83.49240014139272
83.52774832096146
83.56309650053022
83.59844468009896
83.63379285966772
83.66914103923648
83.70448921880524
83.73983739837398
83.77518557794274
83.81053375751148
83.84588193708024
83.881230116649
83.91657829621775
83.9519264757865
83.98727465535525
84.022622834924
8

98.93955461293743
98.97490279250619
99.01025097207494
99.0455991516437
99.08094733121244
99.1162955107812
99.15164369034994
99.1869918699187
99.22234004948746
99.25768822905621
99.29303640862496
99.32838458819371
99.36373276776246
99.39908094733121
99.43442912689996
99.46977730646871
99.50512548603747
99.54047366560623
99.57582184517497
99.61117002474373
99.64651820431247
99.68186638388123
99.71721456344999
99.75256274301873
99.78791092258749
99.82325910215624
99.85860728172499
99.89395546129374
99.9293036408625
99.96465182043124
100.0


In [15]:
#using the inbuilt sklearn Navie bayes for classification
#Importing the Multinomial Navie bayes
from sklearn.naive_bayes import MultinomialNB
#creating a classifier
clf=MultinomialNB()
#calling fit on the classifier with our Numpy 2D array which we have created for training and taget values of training
clf.fit(X_train_modified,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
#here by using the function clf.predict we are predicting the class for test data we are using the Numpy 2D array which we created for testing
Y_pred=clf.predict(X_test_modified)
#importing the confusion matrix and classification report to check the correctness of our prediction
from sklearn.metrics import confusion_matrix,classification_report
#printing the classification_report and confusion matrix for particular classification
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       128
           1       0.65      0.86      0.74       149
           2       0.91      0.53      0.67       156
           3       0.67      0.79      0.72       150
           4       0.75      0.81      0.78       145
           5       0.82      0.81      0.82       148
           6       0.73      0.81      0.77       146
           7       0.84      0.85      0.85       157
           8       0.94      0.92      0.93       157
           9       0.97      0.94      0.95       152
          10       0.96      0.96      0.96       129
          11       0.97      0.89      0.93       140
          12       0.80      0.79      0.80       145
          13       0.94      0.90      0.92       142
          14       0.91      0.93      0.92       147
          15       0.88      0.90      0.89       157
          16       0.91      0.93      0.92       140
          17       0.97    

# Our own Navie's Bayes classifier
#Our own Navie's Bayes classifier contains the 2 function Maininly 
#First one is Fit fuctiontion that return the required dictionary which we require in our prediction
#Second is our predict function that do the predictions for us it has subfunction or Helper function also

In [17]:
#fit funcytion takes the 2 arguments as input that is X_train i.e data matrices (Numpy matrix in 2D format)
#the second argument is target values which are used for training
#it return the dictionary which is to be used in prediction 
def fit(X_train,Y_train):
#we have created the dictionary whicth contains the information in level wise manner in form of keys and values
#The idea here is to firstly store the different classes as keys 
#then for different classes we store what are all words that are present 
#after storing the word we are going to store their count
    result={}
#the different class values is obtained by using the inbuilt set function  over the target values which return only distinct values 
    class_values=set(Y_train)
#here we are storing the total length of the taget this value is used when we do prediction in second term of formula.
#i.e.P('particular class'/'total classes')
    result['total_data']=len(Y_train)
#itterating over each class i.e different target values
    for current_class in class_values:
#we have created a dictionary were we store the different features(words) as keys and their count as values 
        result[current_class]={}
#here we are selecting only those rows we have taget value equal to current class or current target value
#we get a boolean list of True and false from this which we server in X_train and Y_train to get only those rows which have current class as target
        current_class_rows=Y_train==current_class
        X_train_current=X_train[current_class_rows]
        Y_train_current=Y_train[current_class_rows]
#here we are obtaining the number of feature by using shape function which gives number of rows and number of columns
#since we have to only use the number of columns we are giving index as 1
        num_features=X_train.shape[1]
#sum_of total_words this will give us the total number of words in the current row it is intialised by 0 now
        sum_of_total_words=0
#we are itterating over the each feature we have started over iiteration from 1 beacuse it is more intutive.
#i.e. Generally we hear feature 1 not feature 0 and we are going to number of feature+1
        for j in range(1,num_features+1):
#we are storing the sum of word(feature) in particular column
#here j-1 is used because internally the indexing starts from 0 not 1 it is for our sake of simplicity we have taken it from 1
#sum_of_total_words store the total number of words in each class
            result[current_class][j]=(X_train_current[:,j-1]).sum()
            sum_of_total_words+=(X_train_current[:,j-1]).sum()
#we are maintaing a total_count of words for each class
        result[current_class]['total_count']=sum_of_total_words
            
#we are returning the dictionary            
    return result
        

In [18]:
#this function is serve as Helper function for predictSinglePoint
#The idea is we take the test data for each row we find the probability of that row belonging to current class.
#And whose ever class probability is greater we predict that class for particulat row or you can say document
#This function takes 3 arguments as input the dictionary which you get from fit function
#x it represent the particular row in our testing data
#the class for which we are finding the probablity
#Here we using the log probablity instead of Natural one which we find just by dividing
#The idea is since the probability number are quiet small and their multiplication will result in 0 we use log probability 
def probability(dictionary,x,current_class):
#here the output variable initially contain's the second term of our total probability calclulation.
#i.e.P('particular class'/'total classes')
#since we are using the logarithmic probability and in log there is a simple property that division become Multiplication
#we are using that here
    output=np.log(dictionary[current_class]['total_count'])-np.log(dictionary['total_data'])
#we are itterating over the each feture since fromstarting we have saved the dictionary with feature values starting from 1
#so we are itterating in range from 1 to length of all features+1
    for i in range(1,len(x)+1):
#We are only taking those feature for which the count is not 0 ignoring thoose for which count is 0 
#This will save us time 
#instead of i we use i-1 because internally in system indexing starts from 0
        if x[i-1]!=0:
#we are calculating the first term numerator by taking the log of count of word(feature).'1' is added as a laplace correction 
#First term=P('dtest'/y=ci)=P(w1/y=ci)*P(w2/y=ci)*----
            numerator=np.log(dictionary[current_class][i]+1)
#we calculate the denominatortor by taking the value of total Number of words in that particular class
#len(x) is a laplace correction factor that represent the total number of features 
            denominator=np.log(dictionary[current_class]['total_count']+len(x))
#Since we taking log so here i am using the simple propertie of log that division is converted into subtraction
            probability_x=numerator-denominator
#Once again log propertie i sused that multiplication if converted into addition which we are using to calculate total probability
#probability of this x-document belongs to particular class 
            output=output+probability_x
#returning the value of probability
    return output

In [19]:
#this function is a helper function of the predict fuction
#this function used to provide us the best_class values i.e. prediction for a particular document
#we append this prediction in a list in predict function
#This function takes 2 argument.
#The dictionary which you get from the fit function
#And the second argument is x-a single row from X_test Numpy 2D array or you can say particular document
#the intution or idea is we check or calculate probability for each class and whose ever class probablity is higher we predict that class.
def predictSinglePoint(dictionary,x):
#we find the different target values or different classes the dictionary contain using dictionary.keys()
    classes=dictionary.keys()
#Intially we take best_probability value as -10000 and best_class as -1 these value are updated as we go through each class
#To Ensure the values get updated in definetly in first run we are taking a flag as True we make it False after first run 
    best_p=-10000
    best_class=-1
    first_run=True
#itterating over each class
    for current_class in classes:
#Since we have also stored the total_data in dictionary at this level.
#And we don't want to do anything for it we will continue
        if current_class=='total_data':
            continue
#otherwise we calculate probability for current class by calling the probability function
        else:
            p_current_class=probability(dictionary,x,current_class)
#if probablitity is greater than best_p we will update the value of best_p and best_class it is also updated is it is first run
#otherwise go to next class
            if first_run or p_current_class>best_p:
                best_p=p_current_class
                best_class=current_class
#Making the falg value False which we have made True to update for first run
            first_run=False
#At last returning the best_class 
    return best_class

In [20]:
#This fuction is used to predict the classes for the Testing data
#It takes 2 argument as Input
#First argument is dictionary which we get from the fit function.
#And second is test data in specified format(Numpy 2D arary)
def predict(dictionary,X_test):
#we are creatinfg a list in which we will append all our prediction and return this list at the end
    y_pred=[]
#itterating over the Testing_data i.e. going fronm one row to another
    for x in X_test:
#predicting the class for particual document which is presented in form of row in Testing data by clling predictSinglePoint function
        x_class=predictSinglePoint(dictionary,x)
#Appending this Prediction in our list
        y_pred.append(x_class)
#returning the list    
    return y_pred

In [21]:
#Calling the fit Fuction and storing the dictionary return by it in dictionary
#Here we using Numpy 2D array which we have created for training earlier and Target values for training
dictionary=fit(X_train_modified,Y_train)

In [22]:
#we are calling the predict function for doing the predictions 
#here we are passing the dictionary which we get from the fit function and Test_data in Numpy 2D array format
#This fuction takes some time depending on the processor because we are calculating probability for each class for a particular document
#Wait for some time
Y_pred=predict(dictionary,X_test_modified)

In [23]:
#We are printing the classification report and and confusion matrix to check the correctness of our predictions
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.90      0.91      0.90       128
           1       0.64      0.85      0.73       149
           2       0.94      0.47      0.63       156
           3       0.68      0.80      0.73       150
           4       0.74      0.82      0.78       145
           5       0.84      0.83      0.83       148
           6       0.78      0.83      0.80       146
           7       0.82      0.87      0.84       157
           8       0.90      0.90      0.90       157
           9       0.97      0.94      0.95       152
          10       0.97      0.95      0.96       129
          11       0.96      0.89      0.93       140
          12       0.82      0.81      0.82       145
          13       0.95      0.91      0.93       142
          14       0.91      0.93      0.92       147
          15       0.88      0.91      0.90       157
          16       0.90      0.91      0.90       140
          17       0.97    

#Result

In [24]:
# We are getting quiet decent Accuracy.
#In both cases either by using inbuilt Navie's Bayes or our own Navie's Bayes we are getting Approximately same accuracy

In [25]:
#************************************End of Project*****************************************************************